In [1]:
import argparse
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import invgamma, chi2, t

from IPython import display
import pylab as pl
from scipy.signal import lfilter
import mne 
from mne.preprocessing import ICA
import pylab as pl
from IPython.display import display, Latex, Math, clear_output
from IPython import display as display1
import pickle


# parser = argparse.ArgumentParser(description='Compare LD vs ICA')
# parser.add_argument('--r', default=5, type=int, help='Number of sources')
# parser.add_argument('--M', default=8, type=int, help='Number of mixtures')
# parser.add_argument('--N', default=30000, type=int, help='Number of samples')
# parser.add_argument('--NumberofIterations', default=10000, type=int, help='Number of LDInfomax Iterations')
# parser.add_argument('--NumAverages', default=100, type=int, help='Number of Averages')
# parser.add_argument('--SNR', default=40, type=float, help='SNR')
# parser.add_argument('--epsv', default=1e-5, type=float, help='epsilon value')
# parser.add_argument('--outname', default='SIR40Av100script.pickle', type=str, help='Output Name')


r = 5
M = 8
N = 30000
NumberofIterations=10000
NumAverages = 1
SNR = 40
epsv = 1e-5
outname = "SIR40Av100script"

epsvmax = epsv

# args = parser.parse_args()
# # Number of sources
# r=args.r
print('Number of sources:',r)
print(r)
# Number of mixtures
# M=args.M
print('Number of mixtures:',M)
# Number of samples
# N=args.N
print('Number of samples:',N)
# Number of Iterations
# NumberofIterations=args.NumberofIterations
print('Number of iterations:',NumberofIterations)
# Number of Averages
# NumAverages=args.NumAverages
print('Number of Averages:',NumAverages)
# SNR level
# SNR=args.SNR #dB
print('SNR:',SNR)
# Output filename
# outname=args.outname
print('Output filename:',outname)
# Epsilon
# epsvmax=args.epsv
print('Epsilon value:',epsvmax)


NoiseAmp=(10**(-SNR/20))*np.sqrt(r)


#Define number of sampling points
n_samples = N
#Degrees of freedom
df = 4

# Correlation values
rholist=np.array([0.0, 0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
rholist=np.array([0.9])
SIRLDInfoMax=np.zeros((len(rholist),NumAverages))
SINRLDInfoMax=np.zeros((len(rholist),NumAverages))
MSELDInfoMax=np.zeros((len(rholist),NumAverages))
SigPowList=np.zeros((len(rholist),NumAverages))
SIRLDInfoMaxW=np.zeros((len(rholist),NumAverages))
SIRICAInfoMax=np.zeros((len(rholist),NumAverages))
SINRICAInfoMax=np.zeros((len(rholist),NumAverages))
MSEICAInfoMax=np.zeros((len(rholist),NumAverages))
# Nonnegative Clipping function
def nonnegativeclipping(inp):
    out=(inp>0)*((inp)<1.0)*inp+(inp>=1.0)*1.0+(inp<=0)*0.0
    return out

def ProjectOntoLInfty(X):
    return X*(X>=-1.0)*(X<=1.0)+(X>1.0)*1.0-1.0*(X<-1.0)
    
# Calculate SIR Function
def CalculateSIR(H,pH):
    G=pH@H
    Gmax=np.diag(np.max(abs(G),axis=1))
    P=1.0*((np.linalg.inv((Gmax))@np.abs(G))>0.99)
    T=G@P.T
    rankP=np.linalg.matrix_rank(P)
    SIRV=np.linalg.norm((np.diag(T)))**2/(np.linalg.norm(T,'fro')**2-np.linalg.norm(np.diag(T))**2)
    return SIRV,rankP

def CalculateSINR(Out,S):
    r=S.shape[0]
    G=np.dot(Out-np.reshape(np.mean(Out,1),(r,1)),np.linalg.pinv(S-np.reshape(np.mean(S,1),(r,1))))
    indmax=np.argmax(np.abs(G),1)
    GG=np.zeros((r,r))
    for kk in range(r):
        GG[kk,indmax[kk]]=np.dot(Out[kk,:]-np.mean(Out[kk,:]),S[indmax[kk],:].T-np.mean(S[indmax[kk],:]))/np.dot(S[indmax[kk],:]-np.mean(S[indmax[kk],:]),S[indmax[kk],:].T-np.mean(S[indmax[kk],:]))#
    ZZ=GG@(S-np.reshape(np.mean(S,1),(r,1)))+np.reshape(np.mean(Out,1),(r,1))
    E=Out-ZZ
    MSE=np.linalg.norm(E,'fro')**2
    SigPow=np.linalg.norm(ZZ,'fro')**2
    SINR=(SigPow/MSE)
    return SINR,SigPow,MSE,G

########################################################################################
########################################################################################
###                                                                                  ###
###                        SIMULATION                                                ###
###                                                                                  ###
########################################################################################
########################################################################################
pl.figure(figsize=(15, 6), dpi=80)

for iter1 in range(NumAverages):
    iter0=-1
    for rho in rholist:
        epsv=epsvmax*(1.0/(1.0-rho))
        iter0=iter0+1
        calib_correl_matrix = np.eye(r)*(1-rho)+rho*np.ones((r,r))
        mu = np.zeros(len(calib_correl_matrix))
    
        #######################################################
        #              GENERATE SOURCES                       #
        #######################################################
        s = chi2.rvs(df, size=n_samples)[:, np.newaxis] 
        Z = np.random.multivariate_normal(mu, calib_correl_matrix,n_samples)
        X = np.sqrt(df/s)*Z #chi-square method
        # Copula-t distriution with df degrees of freedom
        U = t.cdf(X,df).T
        # Generate correlated unity variance sources
        S=U#*np.sqrt(12)
        S = 2*S -1
        
        
        #######################################################
        #              GENERATE MIXINGS                       #
        #######################################################
        # Generate Mxr random mixing from i.i.d N(0,1)
        H=np.random.randn(M,r)
        # Mixtures
        X=np.dot(H,S)
        X=X+NoiseAmp*np.random.randn(X.shape[0],X.shape[1])
        
        #######################################################
        #             LD InfoMax Algorithm                    #
        #######################################################
        # ALGORTITHM STATE INITIALIZATION
        W=np.eye(r,M)*2
        W=np.random.randn(r,M)
        delWo=np.zeros((r,M))
        Wu=np.zeros((r,M))
        # SIR list
        SIRlist=np.zeros(NumberofIterations)
        SIRflist=np.zeros(NumberofIterations)
        erlist=np.zeros(N)
        erflist=np.zeros(N)
        Belist=np.zeros(N)
        SIRflisto=0
        erflisto=0
        Z=(np.random.rand(r,N))/2.0
        RcX=np.dot(X,X.T)/N
        muX=np.mean(X,axis=1).reshape(M,1)
        RX=RcX-np.dot(muX,muX.T)
        RXinv=np.linalg.pinv(RX+epsv*np.eye(M))
        muv=1e2
        DISPLAY_PERIOD=200
        DelZo=np.zeros((r,N))
        
        
        # MAIN ALGORITHM LOOP
        plt.figure(figsize = (10,10))
        SIRlist = []
        for k in range(NumberofIterations):
            if (k>3e3):
                epsv=epsvmax
            RcZ=np.dot(Z,Z.T)/N
            muZ=np.mean(Z,axis=1).reshape(r,1)
            RZ=RcZ-np.dot(muZ,muZ.T)+epsv*np.eye(r)
            Zbar=Z-muZ
            E=Zbar-np.dot(W,(X-muX))
            RcE=np.dot(E,E.T)/N
            muE=np.mean(E,axis=1).reshape(r,1)
            RE=RcE-np.dot(muE,muE.T)+epsv*np.eye(r)
            DelZ=np.dot(np.linalg.pinv(RZ),Z-muZ)/N-np.dot(np.linalg.pinv(RE),E-muE)/N
            Upd=DelZ-DelZo
            Zu=Z+muv*Upd/np.sqrt(k+1)
            Z=nonnegativeclipping(Zu)
#             Z=ProjectOntoLInfty(Zu)
            if (np.mod(k,1)==0):
                RcZX=np.dot(Z,X.T)/N
                muZ=np.mean(Z,axis=1).reshape(r,1)
                RZX=RcZX-np.dot(muZ,muX.T)
                W=np.dot(RZX,RXinv)
            SIRlist.append(10*np.log10(CalculateSINR(Z,S)[0]))
            pl.clf()
            pl.plot(np.array(SIRlist), linewidth = 3)
            pl.xlabel("Number of Iterations / {}".format(1), fontsize = 15)
            pl.ylabel("SIR (dB)", fontsize = 15)
            pl.title("SIR Behaviour", fontsize = 15)
            pl.grid()
            clear_output(wait=True)
            display(pl.gcf()) 
            #SIRv,rankP=CalculateSIR(H,W)
            #SIRlist[k]=SIRv 
            lambdasirf=1-1/10000;
           # SIRflist[k]=lambdasirf*SIRflisto+(1-lambdasirf)*10*np.log10(SIRv)
           # if (np.abs(10*np.log10(SIRv)-SIRflisto)>SIRflisto):
            #    SIRflist[k]=10*np.log10(SIRv)
             #   SIRflisto=SIRflist[k];
        # END OF MAIN ALGORITHM LOOP
        Gld=np.dot(Z-np.reshape(np.mean(Z,1),(r,1)),np.linalg.pinv(S-np.reshape(np.mean(S,1),(r,1))))
        SIRv,rankP=CalculateSIR(Gld,np.eye(r))
        SIRLDInfoMax[iter0,iter1]=SIRv
        SIRv,rankP=CalculateSIR(H,W)
        SIRLDInfoMaxW[iter0,iter1]=SIRv
        SINRv,SigPow,MSEv,G=CalculateSINR(Z,S)
        SINRLDInfoMax[iter0,iter1]=SINRv
        MSELDInfoMax[iter0,iter1]=MSEv
        SigPowList[iter0,iter1]=SigPow



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/bbozkurt15/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_437492/2396372185.py", line 217, in <module>
    display(pl.gcf())
  File "/home/bbozkurt15/anaconda3/lib/python3.9/site-packages/IPython/core/display.py", line 320, in display
    format_dict, md_dict = format(obj, include=include, exclude=exclude)
  File "/home/bbozkurt15/anaconda3/lib/python3.9/site-packages/IPython/core/formatters.py", line 180, in format
    data = formatter(obj)
  File "/home/bbozkurt15/anaconda3/lib/python3.9/site-packages/decorator.py", line 232, in fun
    return caller(func, *(extras + args), **kw)
  File "/home/bbozkurt15/anaconda3/lib/python3.9/site-packages/IPython/core/formatters.py", line 224, in catch_format_error
    r = method(self, *args, **kwargs)
  File "/home/bbozkurt15/anaconda3/lib/python3.9/site-packages/IPython/cor

TypeError: object of type 'NoneType' has no len()

<Figure size 1200x480 with 0 Axes>

Error in callback <function flush_figures at 0x7fc681d8a790> (for post_execute):


KeyboardInterrupt: 

In [2]:
Z.max(), Z.min()

(1.0, 0.0)

In [3]:
S.max(), S.min()

(0.9999835279312468, -0.9999580078248935)

In [4]:
CalculateSIR(H,W)

(20.33962374719415, 5)

In [6]:
10*np.log10(CalculateSINR(Z,S)[0])

26.53411234537093